In [1]:
!pip install ucimlrepo
!pip install matplotlib seaborn
# !pip install sklearn Already preinstalled
!pip install alibi
!pip install shap



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


# Cervical cancer risk factor prediction
We consider the cervical cancer dataset contains indicators and risk factors for predicting whether a woman will get cervical cancer. The features include demographic data (such as age), lifestyle, and medical history. The objectives are: 1) to train a risk predictor (i.e., binary classifier) of cervical cancer, given the input features. 2) to identify the most important risk factors.

The features are:
1.	Age in years
2.	Number of sexual partners
3.	First sexual intercourse (age in years)
4.	Number of pregnancies
5.	Smoking (in years)
6.	Hormonal contraceptives (in years)
7.	Number of years with an intrauterine device (IUD)
8.	Has patient ever had a sexually transmitted disease (STD) yes or no
9.	Number of STD diagnoses
10.	Time since first STD diagnosis
11.	Time since last STD diagnosis
12.	hPV diagnostic

The target is the biopsy results: “Healthy” or “Cancer”.


## Download dataset, partition into train/test

In [1]:
from ucimlrepo import fetch_ucirepo
cervical_cancer = fetch_ucirepo(name = 'Cervical Cancer')

X = cervical_cancer.data.features
y = X['Biopsy'] # Ground truth diagnosis: Biopsy result

# access metadata
print('Number of instances', cervical_cancer.metadata.num_instances)
print('Summary', cervical_cancer.metadata.additional_info.summary)

# access variable info in tabular format
print('All variables' , cervical_cancer.variables)

# Retain only a fraction of the features:

included_features = ['Age',
                     'Number of sexual partners',
                     'First sexual intercourse',
                     'Num of pregnancies',
                     'Smokes (years)',
                     'Hormonal Contraceptives (years)',
                     'IUD (years)',
                     'STDs',
                     'STDs: Number of diagnosis',
                     'STDs: Time since first diagnosis',
                     'STDs: Time since last diagnosis',
                     'Dx:HPV']

X = X[included_features]

Number of instances 858
Summary The dataset was collected at 'Hospital Universitario de Caracas' in Caracas, Venezuela. The dataset comprises demographic information, habits, and historic medical records of 858 patients. Several patients decided not to answer some of the questions because of privacy concerns (missing values).
All variables                                   name     role        type demographic  \
0                                  Age  Feature     Integer         Age   
1            Number of sexual partners  Feature  Continuous       Other   
2             First sexual intercourse  Feature  Continuous        None   
3                   Num of pregnancies  Feature  Continuous        None   
4                               Smokes  Feature  Continuous        None   
5                       Smokes (years)  Feature  Continuous        None   
6                  Smokes (packs/year)  Feature  Continuous        None   
7              Hormonal Contraceptives  Feature  Continuou

## Data cleaning.
Here, we display summary statistics and identify potential issues. Here, we find no aberrant values, but some features are missing for some instances; we replace missing feature values by the median of the dataset.

In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer
print('Before imputation')
print(X.describe())

imputer = SimpleImputer(strategy="median")
X = pd.DataFrame(imputer.fit_transform(X),columns=X.columns)
print('after imputation')
print(X.describe())

Before imputation
              Age  Number of sexual partners  First sexual intercourse  \
count  858.000000                 832.000000                851.000000   
mean    26.820513                   2.527644                 16.995300   
std      8.497948                   1.667760                  2.803355   
min     13.000000                   1.000000                 10.000000   
25%     20.000000                   2.000000                 15.000000   
50%     25.000000                   2.000000                 17.000000   
75%     32.000000                   3.000000                 18.000000   
max     84.000000                  28.000000                 32.000000   

       Num of pregnancies  Smokes (years)  Hormonal Contraceptives (years)  \
count          802.000000      845.000000                       750.000000   
mean             2.275561        1.219721                         2.256419   
std              1.447414        4.089017                         3.764254   
min

## Data Partition

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,stratify=y)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(643, 12) (215, 12) (643,) (215,)


---

---

## B. Logistic Regression

### 1.	Train an L2-regularized logistic regression model on the training data set with an optimal using the AUCROC metric. You can use the sklearn.linear_model.LogisticRegressionCV function to automatically adjust the value of the L2 penalty.

### 2.	Report the classification performance on the train and test set (accuracy, AUROC, and negative log-likelihood).

### 3.	Calculate the feature importance (defined as the standard deviation of the feature effects) and visualize them as a bar plot.

### 4.	What are the most and least important features?

---

## C. Generalized Additive Model
The log-odds ratio of having cervical cancer is not expected to be linearly related to the numerical features. Hence, a Generalized Additive Model could be more accurate.

### 1.	Build and train a Generalized Additive Model, where the numerical features have a trainable, non-linear effect, and the others have a linear effect. 
Your model should be implemented as a scikit-learn Pipeline (sklearn.pipeline.Pipeline), where numerical features are transformed via B-splines (cubic order, 5 knots, constant extrapolation; use sklearn.preprocessing.SplineTransformer) while other features are not transformed (use sklearn.compose.ColumnTransformer), followed by an L2-regularized logistic regression model (use sklearn.linear_model.LogisticRegressionCV).

### 2.	Report the classification performance on the train and test set (accuracy, AUROC, and negative log-likelihood) and compare with the performance of the logistic regression model.

### 3.	Visualize the learnt non-linearity (use sklearn.inspection.partial_dependence).

### 4.	Calculate the feature effects (you can adapt the code snippet from the tutorial).

### 5.	What are the most and least important features? Comment on the differences.

---

## D. Black-box Classifier Model
A model taking into account interactions between features could yield better predictive performance at the cost of reduced interpretability. We will build such a black-box model and use a-posteriori interpretation/explanation methods.

### 1.	Train a Random Forest classifier (sklearn.ensemble.RandomForestClassifier) with n_estimators=200 trees. 
Optimize the min_samples_leaf hyperparameter (from 1 to 100) using cross-validation over the train set (use sklearn.model_selection.GridSearchCV and sklearn.model_selection.KFold).

### 2.	Determine the feature importance using the permutation importance metric on the train and test set and visualize them (sklearn.inspection.permutation_importance).

### 3.	Conclude on the most important features, and on the features for which overfitting occurs. Are the conclusions different from previously? Why?

### 4.	Using the alibi package (Accumulated Local Effects (ALE)), plot the Accumulated Local Effects for all numerical features and report them. How do the ALE plots compare with the partial dependence plots of the GAM model?

---

## E. Shapley Values

Let us now try to compare how each model relies on each feature, and explain individual predictions. Using the TreeExplainer class of the shap package, calculate the Shapley values of the Random Forest model over the test set.

### 1.	Visualize the Shapley values using a summary plot. How do they compare to ALE plots?

### 2.	Calculate the Shapley feature importance, as the average of the absolute value of the Shapley values. How do they compare to the feature importances determined in D.2?

### 3.	Pick two test set instances for which the Random Forest model makes an incorrect prediction. Explain their corresponding prediction using a Shapley values force plot.

## F. Bonus

Based on the various model interpretations provided, can you come up with a better model, based on a different set of features?